In [11]:
%cd ..
%rm -rf no-time-to-backprop

/content


In [1]:
!ls
!nvidia-smi

sample_data
Thu Dec 18 21:20:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------

In [2]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.80"

!git clone https://github.com/Francis2002/no-time-to-backprop.git
%cd no-time-to-backprop

Cloning into 'no-time-to-backprop'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 231 (delta 59), reused 62 (delta 32), pack-reused 138 (from 1)
Receiving objects: 100% (231/231), 91.73 MiB | 22.94 MiB/s, done.
Resolving deltas: 100% (128/128), done.
/content/no-time-to-backprop


In [3]:
!python -V
!ls

Python 3.12.12
constraints.txt			 requirements-macos-cpu.txt  run_on_gpu.ipynb
hpt.py				 requirements.mac.txt	     run_toy_task.py
plotting_scripts		 requirements.txt	     test_rec.py
requirements-base.txt		 run_all_benchmarks.sh	     tgap
requirements.from_linux_env.txt  run_all.sh
requirements-linux-cuda.txt	 run_all_toy.sh


In [4]:
%pip install -r requirements-linux-cuda.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.2/488.2 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 113.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 116.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 136.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [6]:
import jax
import jaxlib
import sys

print(f"jax: {jax.__version__}")
print(f"jaxlib: {jaxlib.__version__}")
print(f"executable: {sys.executable}")
print(f"devices: {jax.devices()}")


jax: 0.8.1
jaxlib: 0.8.1
executable: /usr/bin/python3
devices: [CudaDevice(id=0)]


In [7]:
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --task link_classification

[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/mooc_ONLINE
[*] d_model set to 64
Running iteration: 0
[*] Output file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists. Using existing file.
object address  : 0x7bd2cbad2f20
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [ ]:
# Here we run experiments
!python3 run_toy_task.py -m ONLINE -a ZUC --num_layers 4 --num_hidden 64 --double_dmodel --mixing rotational_full --dataset mooc --batching_strategy rearranged --lr 0.001 --rec_learning_factor 0.5 --dropout 0.1 --weight_decay 0.1 --batch_size 50 --num_epochs 100 --num_gradient_accumulation_steps 16 --dedupe

: 